In [1]:
import boto3

In [10]:
%%writefile Dockerfile

FROM python:3.9-slim-buster

COPY requirements.txt /opt/ml/requirements.txt
COPY code/preprocess.py /opt/ml/code/preprocess.py
COPY code/train.py /opt/ml/code/train.py

RUN pip3 install -r /opt/ml/requirements.txt

ENV SAGEMAKER_PROGRAM preprocess.py
ENV SAGEMAKER_PROGRAM train.py
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

Overwriting Dockerfile


In [11]:
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
ecr_repository = 'demo_prophet'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

In [12]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository .

Sending build context to Docker daemon  308.7kB
Step 1/9 : FROM python:3.9-slim-buster
 ---> c84dbfe3b8de
Step 2/9 : COPY requirements.txt /opt/ml/requirements.txt
 ---> Using cache
 ---> 19967f0ddabd
Step 3/9 : COPY code/preprocess.py /opt/ml/code/preprocess.py
 ---> Using cache
 ---> c833d36c2ed6
Step 4/9 : COPY code/train.py /opt/ml/code/train.py
 ---> Using cache
 ---> 4237e55159b7
Step 5/9 : RUN pip3 install -r /opt/ml/requirements.txt
 ---> Using cache
 ---> b01a2f7888c6
Step 6/9 : ENV SAGEMAKER_PROGRAM preprocess.py
 ---> Using cache
 ---> a5d20bfc0bff
Step 7/9 : ENV SAGEMAKER_PROGRAM train.py
 ---> Using cache
 ---> 82820ae4ae4d
Step 8/9 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> f49b0f761987
Step 9/9 : ENTRYPOINT ["python3"]
 ---> Running in d4a87b4650ac
Removing intermediate container d4a87b4650ac
 ---> 721d0d4cb60f
Successfully built 721d0d4cb60f
Successfully tagged demo_prophet:latest


In [13]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [14]:
!aws ecr create-repository --repository-name $ecr_repository


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'demo_prophet' already exists in the registry with id '654654222480'


In [15]:
# aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
# if [ $? -ne 0 ]
# then
# aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
# fi

In [16]:
!docker tag {ecr_repository + tag} $processing_repository_uri

In [17]:
!docker push $processing_repository_uri

The push refers to repository [654654222480.dkr.ecr.us-east-1.amazonaws.com/demo_prophet]

ab15a0ce: Preparing 
86d10eee: Preparing 
7fda46cc: Preparing 
a3b6f7dc: Preparing 
a27560c1: Preparing 
037e08b3: Preparing 
eede8d6e: Preparing 
55769c5e: Preparing 
8a51359d: Layer already exists latest: digest: sha256:50085beb34e9e6337603696bd62671c4637fcc7993c9a5d2e40692a926620bae size: 2204
